# Analiza ofert wynajmu mieszkań we Wrocławiu na podstawie danych serwisu OTODOM
## Problem Biznesowy
Problem biznesowy który zdecydowaliśmy się przeanalizować, polegał na sprawdzeniu, które z następujących udogodnień (bądź ich kombinacji) jest najbardziej opłacalna dla potencjalnego własciciela, przy wynajmie mieszkania; Które z następujących udogodnień, najbardziej podnosi cenę najmu mieszkania:
* Zmywarka
* Lodowka
* Meble
* Piekarnik
* Kuchenka
* Pralka
* Telewizor
* Telewizja kablowa
* Internet
* Telefon

## Metoda analizy
Dane zostały przeanalizowane przez model regresji liniowej, używając modelu liniowego z biblioteki *scikit-learn* - model oblicza współczynnik zmiany ceny. Dane zostały wczytane z pliku .csv do struktury *dataframe* z biblioteki *pandas*. W ramach analizy przeanalizowaliśmy następująco:
* Wzrost ceny w zależności od udogodnienia.
* Wzrost ceny za $m^2$ w zależności od udogodnienia.
* Wzrost ceny w zależności od kombinacji 2 udogodnień.
* Wzrost ceny za $m^2$ w zależności od kombinacji 2 udogodnień.

W celu analizy ceny za metr kwadratowy do ramki danych dodaliśmy kolumnę *cena_za_m2*. Z ramki danych usunęliśmy wszystkie pozycje które mogły miec pustą cenę/ powierzchnię. Stwierdziliśmy że warto także zliczać ilość występowania udogodnień oraz ich kombinacji.

In [1]:
import pandas as pd
import itertools
from sklearn.linear_model import LinearRegression

df = pd.read_csv("otodom_all.csv", sep=';', engine='python')

df = df.dropna(subset=['cenaPLN', 'powierzchnia_mkw'])
df['cena_za_m2'] = df['cenaPLN'] / df['powierzchnia_mkw']

features = [
    'zmywarka', 'lodowka', 'meble', 'piekarnik', 'kuchenka',
    'pralka', 'telewizor', 'telewizja_kablowa', 'internet', 'telefon'
]

freq = {}
for feature in features:
    freq[feature] = df[feature].sum()

freq_df = pd.DataFrame.from_dict(freq, orient='index', columns=['wystąpienia'])
freq_df = freq_df.sort_values(by='wystąpienia', ascending=False)
freq_df

,wystąpienia
meble,3878
lodowka,3260
pralka,3111
kuchenka,2915
piekarnik,2730
zmywarka,2344
internet,2229
telewizor,1602
telewizja_kablowa,1262
telefon,477


### Kombinacje udogodnień
Istnieje $\left( \begin{array}{c} 10 \\ 2 \end{array} \right) = \frac{10!}{2!(10-2)!} = \frac{10 * 9}{2} = 45$ kombinacji dwóch udogodnień. Żeby nie musieć spisywać ręcznie każdej z nich, uzyliśmy biblioteki *itertools* w celu wygenerowania wszystkich kombinacji automatycznie. Z każdej z nich powstała nowa kolumna zawierająca wynik mnożenia boolowskiego: 

***df['telefon_telewizor']*** = ***df['telefon']*** * ***df['telewizor']***

Pozwala to uzyskać współczynnik zmiany dla mieszkań które zawierają taką kombinację udogodnień. Najlepsze wyniki uzyskuje się licząc oddzielnie 4 warianty przypadków, wymienionych wyżej.


In [2]:
pairs = list(itertools.combinations(features, 2))

pair_freq = {}
for f1, f2 in pairs:
    colname = f"{f1}_x_{f2}"
    df[colname] = df[f1] * df[f2]
    pair_freq[colname] = df[colname].sum()

freq_df = pd.DataFrame.from_dict(pair_freq, orient='index', columns=['wystąpienia'])
freq_df = freq_df.sort_values(by='wystąpienia', ascending=False)
freq_df

,wystąpienia
lodowka_x_meble,3158
lodowka_x_pralka,3096
meble_x_pralka,3035
lodowka_x_kuchenka,2905
meble_x_kuchenka,2847
kuchenka_x_pralka,2804
lodowka_x_piekarnik,2693
meble_x_piekarnik,2637
piekarnik_x_pralka,2602
piekarnik_x_kuchenka,2598


## Dane ofert
Dane na podstawie których robiliśmy analizę zawierały wiele nieużytych przez nas kolumn. Wszystkie użyte przez nas kolumny zawierały tylko wartości liczbowe - łatwe do zrozumienia przez prymitywny model nauczania maszynowego, którego użyliśmy w celu wykonania tej analizy. Udogodnienia oznaczone są 0 w przypadku gdy ich nie ma, oraz 1 w przypadku gdy są. Inne użyte przez nas kolumny (czynsz, metraż w $m^2$).

## Wpływ udogodnień na cenę


In [3]:
X = df[features]
y = df['cenaPLN']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'udogodnienie': X.columns,
    'wplyw_na_cene': model.coef_,
    'liczba_wystapien': [freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene', ascending=False)
results

,udogodnienie,wplyw_na_cene,liczba_wystapien
0,zmywarka,659.248632,2344
6,telewizor,306.666701,1602
8,internet,223.767328,2229
7,telewizja_kablowa,130.115195,1262
3,piekarnik,-65.108974,2730
9,telefon,-116.041124,477
5,pralka,-207.062346,3111
4,kuchenka,-258.584413,2915
1,lodowka,-402.037340,3260
2,meble,-449.099941,3878


Udogodnieniem najbardziej wpływającym na czynsze okazała się zmywarka, a najbardziej szkodliwym - lodówka. Najrzadszym z udogodnień okazał się telefon - może sprawić że wyniki będą bardzo przechylone w którymś kierunku - szczególnie w przypadku kombinacji.

## Wpływ udogodnień na cenę za $m^2$

In [4]:
X = df[features]
y = df['cena_za_m2']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'udogodnienie': X.columns,
    'wplyw_na_cene_m2': model.coef_,
    'liczba_wystapien': [freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene_m2', ascending=False)
results

,udogodnienie,wplyw_na_cene_m2,liczba_wystapien
1,lodowka,6.308641,3260
0,zmywarka,4.145745,2344
8,internet,3.568490,2229
6,telewizor,2.952334,1602
4,kuchenka,2.560517,2915
7,telewizja_kablowa,0.892410,1262
2,meble,-1.902229,3878
9,telefon,-2.576766,477
5,pralka,-3.831876,3111
3,piekarnik,-10.622430,2730


Odwrotną obserwację, w porównaniu do poprzednich można zobaczyć, gdy będziemy analizowac nie cenę, ale cenę za $m^2$. W tym wypadku okazuje się że to lodówka jest najbardziej opłacalnym udogodnieniem, w jakie można zainwestować, a zmywarka zajmuje 2 miejsce. Piekarnik przynosi dużą szkodę ponad 10 złotych na metr kwadratowy do czynszu. Takie wyniki mogą wynikać z częstszego występowania lodówek, szczególnie w przypadku mieszkań z niższymi czynszami.

## Wpływ kombinacji udogodnień na cenę

In [5]:
X = df[[f"{f1}_x_{f2}" for f1, f2 in pairs]]
y = df['cenaPLN']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'para': X.columns,
    'wplyw_na_cene': model.coef_,
    'liczba_wystapien': [pair_freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene', ascending=False)
results

,para,wplyw_na_cene,liczba_wystapien
38,pralka_x_telefon,1959.689328,302
42,telewizja_kablowa_x_internet,973.366872,1168
19,meble_x_pralka,957.169824,3035
35,pralka_x_telewizor,953.278460,1560
15,lodowka_x_internet,885.310188,1689
0,zmywarka_x_lodowka,551.138990,2312
3,zmywarka_x_kuchenka,479.990937,2204
14,lodowka_x_telewizja_kablowa,446.744221,903
29,piekarnik_x_telefon,310.584286,280
7,zmywarka_x_internet,288.383989,1226


Jak zostało wspomniane wcześniej, telefon może przechylać wyniki, w obie strony, z powodu małej liczby wystąpień. Najbardziej opłacalnymi kombinacjami udogodnień są:
* Pralka i Telefon
* Telewizja kablowa i internet
* Meble i Pralka
* Pralka i Telewizor
* Lodówka i Internet

A takie połączenia jak lodówka z telefonem, bądź meblami są w stanie spowodować dużą stratę na czynszu.

## Wpływ kombinacji udogodnień na cenę za $m^2$

In [6]:
X = df[[f"{f1}_x_{f2}" for f1, f2 in pairs]]
y = df['cena_za_m2']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'para': X.columns,
    'wplyw_na_cene_m2': model.coef_,
    'liczba_wystapien': [pair_freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene_m2', ascending=False)
results

,para,wplyw_na_cene_m2,liczba_wystapien
29,piekarnik_x_telefon,26.067584,280
15,lodowka_x_internet,23.868730,1689
20,meble_x_telewizor,11.974141,1592
11,lodowka_x_kuchenka,11.640692,2905
19,meble_x_pralka,7.996518,3035
25,piekarnik_x_pralka,7.754759,2602
42,telewizja_kablowa_x_internet,7.719167,1168
36,pralka_x_telewizja_kablowa,6.239043,876
9,lodowka_x_meble,5.614774,3158
26,piekarnik_x_telewizor,4.507348,1348


W przypadku sprawdzania cen za $m^2$ sytuacja opłacalności się zmienia: najbardziej opłacalne okazują się połączenia mebli z pralką albo telewizorem, oraz lodówki z internetem albo kuchenką. Najbardziej opłacalna okazała siępara piekarnika i telefonu. Zaskakującą jest ilość połączeń lodówki, które przynoszą stratę.

## Wpływ dzielnic na cenę udogodnienia

In [7]:
dzielnice_encoded = pd.get_dummies(df['dzielnica'])

interaction_cols = []
for feature in features:
    new_cols = []
    for dzielnica in df['dzielnica'].unique():
        col_name = f"{dzielnica}_x_{feature}"
        mask = (df['dzielnica'] == dzielnica)
        df[col_name] = df[feature] * mask.astype(int)
        if df[col_name].sum() > 50:
            interaction_cols.append(col_name)
        else:
            df.drop(columns=[col_name], inplace=True)

X = df[interaction_cols]
y = df['cenaPLN']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'interakcja': X.columns,
    'wplyw_na_cene': model.coef_,
    'liczba_wystapien': [df[col].sum() for col in X.columns]
}).sort_values(by='wplyw_na_cene', ascending=False)

results

,interakcja,wplyw_na_cene,liczba_wystapien
3,Stare Miasto_x_zmywarka,917.826712,448
2,Śródmieście_x_zmywarka,819.819317,331
0,Fabryczna_x_zmywarka,604.702386,398
4,Krzyki_x_zmywarka,570.134713,788
16,Psie Pole_x_piekarnik,414.958195,412
34,Krzyki_x_telewizor,401.606036,505
27,Śródmieście_x_pralka,385.858140,463
44,Krzyki_x_internet,377.300692,764
1,Psie Pole_x_zmywarka,334.207726,363
33,Stare Miasto_x_telewizor,305.148491,328


Oraz cena za m^2 dla dzielnic i poszczególnych udogodnień:

In [8]:
X = df[interaction_cols]
y = df['cena_za_m2']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'interakcja': X.columns,
    'wplyw_na_cene': model.coef_,
    'liczba_wystapien': [df[col].sum() for col in X.columns]
}).sort_values(by='wplyw_na_cene', ascending=False)

results

,interakcja,wplyw_na_cene,liczba_wystapien
9,Krzyki_x_lodowka,11.766152,1073
8,Stare Miasto_x_lodowka,9.496360,606
21,Psie Pole_x_kuchenka,9.185450,440
7,Śródmieście_x_lodowka,6.420266,486
20,Fabryczna_x_kuchenka,6.077216,531
43,Stare Miasto_x_internet,5.812144,454
4,Krzyki_x_zmywarka,5.275012,788
32,Śródmieście_x_telewizor,5.138508,248
12,Śródmieście_x_meble,4.669442,585
2,Śródmieście_x_zmywarka,4.377496,331


Wyniki zostałe obliczone tylko dla kombinacji które mają przynajmniej 100 wystąpień. W przypadku gdy niektóre z nich mają tylko kilka wystąpień, wyniki stają się niewymierne i osiągają absurdalne wartości (np. piekarnik_Siechnice - 1.566843e+16).

Najbardziej opłacalnym w wielu dzielnicach okazała się zmywarka - która dominuje w 4 pierwszych najbardziej opłacalnych kombinacji dzielnic i udogodnień.
Kiedy weźmiemy pod uwagę wielkość mieszkania, zdecydowanym zwycięzcą zostaje lodówka.

## Podsumowanie

Na podstawie przeprowadzonej analizy ofert wynajmu mieszkań we Wrocławiu na podstawie danych z serwisu OTODOM można wyciągnąć następujące wnioski:

### Wpływ pojedynczych udogodnień na cenę:
- Zmywarka okazała się najbardziej opłacalnym udogodnieniem, które znacząco podnosi cenę najmu mieszkania.
- Lodówka, mimo swojej powszechności, zaniża cenę najmu, co może wynikać z jej częstszego występowania w mieszkaniach o niższym standardzie.

### Wpływ pojedynczych udogodnień na cenę za m²:
- W przypadku analizy ceny za m², lodówka jest najbardziej opłacalnym udogodnieniem, co może wynikać z jej częstszego występowania w mniejszych mieszkaniach, które są bardziej nowoczesne i lepiej wyposażone.
- Piekarnik przynosi największą stratę w cenie za m², co może być związane z jego obecnością w starszych lub mniej atrakcyjnych mieszkaniach.

### Wpływ kombinacji udogodnień na cenę:
- Najbardziej opłacalne kombinacje udogodnień to pralka z telefonem, telewizja kablowa z internetem, oraz meble z pralką. Te połączenia znacząco podnoszą cenę najmu.
- Kombinacje lodówki z telefonem lub meblami powodują największe straty na czynszu, co może wynikać z ich częstszego występowania w mieszkaniach o niższym standardzie.

### Wpływ kombinacji udogodnień na cenę za m²:
- Najbardziej opłacalne kombinacje dla ceny za m² to meble z pralką lub telewizorem, oraz lodówka z internetem lub kuchenką.
- Zaskakująco, wiele kombinacji z lodówką przynosi stratę, co może wskazywać na jej obecność w mieszkaniach o niższej jakości.

### Wpływ dzielnic na cenę udogodnień:
- Najbardziej opłacalne udogodnienia często powtarzają się, w 4 różnych dzielnicach najlepszym udogodnieniem jest zmywarka, a jeśli weźmiemy pod uwagę rozmiar mieszkania, 3-krotnie w top 5 powtarza się lodówka.
- Niektóre dzielnice (np. Rynek) są generalnie bardziej opłacalne, bez względu na udogodnienie, niż inne. jednak zaskakująco wciąż większe znaczenie maja udogodnienia.

### Ogólne wnioski:
- Udogodnienia takie jak zmywarka, pralka, czy internet mają największy pozytywny wpływ na cenę najmu, zarówno w ujęciu całkowitym, jak i za m².
- Lodówka, mimo swojej powszechności, ma zróżnicowany wpływ – zaniża cenę najmu, ale podnosi cenę za m², co wskazuje na jej różne znaczenie w zależności od typu mieszkania.
- Kombinacje udogodnień mogą znacząco zmieniać opłacalność – warto inwestować w połączenia, które są atrakcyjne dla najemców, np. pralka z meblami lub internet z telewizją kablową.
- Analiza pokazuje, że decyzje dotyczące wyposażenia mieszkania powinny być podejmowane w kontekście jego standardu, lokalizacji oraz docelowej grupy najemców.